<a href="https://colab.research.google.com/github/Fush1m1/211214sample/blob/main/signate220319beginer%E9%99%90%E5%AE%9A%E3%82%B3%E3%83%B3%E3%83%9A%EF%BC%88%E7%B3%96%E5%B0%BF%E7%97%85%EF%BC%89.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データとライブラリの読み込み

In [59]:
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [60]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sub = pd.read_csv('sample_submit.csv',header=None)

# クレンジングと特徴量の作成

In [61]:
train = train.drop("index", axis=1)
test = test.drop("index", axis=1)

# モデル構築

In [62]:
X_train = train.drop(['Outcome'], axis=1)
y_train = train['Outcome']

In [63]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

#X_trainとy_trainをtrainとvalidに分割
train_x, valid_x, train_y, valid_y = train_test_split(X, y, test_size=0.33, random_state=0)

#LightGBMの分類器をインスタンス化
gbm = lgb.LGBMClassifier(objective='binary')

#trainとvalidを指定し学習
gbm.fit(train_x, train_y, eval_set = [(valid_x, valid_y)],
        early_stopping_rounds=20, #20回連続でlossが下がらなかったら終了
        verbose=10  #10round毎に、lossを表示
)


Training until validation scores don't improve for 20 rounds.
[10]	valid_0's binary_logloss: 0.490868
[20]	valid_0's binary_logloss: 0.473108
[30]	valid_0's binary_logloss: 0.46972
[40]	valid_0's binary_logloss: 0.468288
[50]	valid_0's binary_logloss: 0.467052
Early stopping, best iteration is:
[37]	valid_0's binary_logloss: 0.466403


LGBMClassifier(objective='binary')

In [64]:
#valid_xについて推論
#out of fold　=　学習に使っているデータ以外の予測
oof = gbm.predict(valid_x, num_iteration=gbm.best_iteration_)
#正解率
print('score', round(accuracy_score(valid_y, oof)*100,2), '%')

score 78.79 %


In [65]:
#testの予測
test_pred = gbm.predict(test, num_iteration=gbm.best_iteration_)
sub[1] = test_pred  # sample_submitの列1をtest_predに置き換え